In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install pandas \
            sentence-transformers \
            faiss-cpu \
            transformers \
            onnx \
            onnxruntime \
            torch

!pip install optimum[onnxruntime]
!pip install openai-whisper \
            torchaudio \
            ffmpeg-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 95.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

In [6]:
# ============================================================
# ✅ STEP 1: Environment Setup
# ============================================================


In [7]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Avoids tokenizer warnings
os.environ["OMP_NUM_THREADS"] = "1"  # Limits threads for better performance on small machines


In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
import onnx
import onnxruntime as ort


In [9]:
# ============================================================
# ✅ STEP 2: Load Model & Tokenizer
# ============================================================


In [11]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"✅ Loaded model on {device}")


model.safetensors:   0%|          | 0.00/539M [00:00<?, ?B/s]

✅ Loaded model on cuda


In [13]:
# ============================================================
# ✅ STEP 3: Prepare Dummy Input
# ============================================================
input_text = "set an alarm"
inputs = tokenizer(input_text, return_tensors="pt", padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}


In [14]:
onnx_path = "embedding_model.onnx"

torch.onnx.export(
    model,
    (inputs["input_ids"], inputs["attention_mask"]),
    onnx_path,
    input_names=["input_ids", "attention_mask"],
    output_names=["last_hidden_state"],
    dynamic_axes={
        "input_ids": {0: "batch_size"},
        "attention_mask": {0: "batch_size"},
        "last_hidden_state": {0: "batch_size"},
    },
    opset_version=14,
    do_constant_folding=True,
)


/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask


In [15]:
# ============================================================
# ✅ STEP 5: Verify Exported Model
# ============================================================
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)
print("✅ ONNX model structure verified successfully")


✅ ONNX model structure verified successfully


In [16]:
# ============================================================
# ✅ STEP 6: Run Inference Using ONNX Runtime
# ============================================================
input_ids = inputs["input_ids"].cpu().numpy()
attention_mask = inputs["attention_mask"].cpu().numpy()

session = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])

onnx_inputs = {
    "input_ids": input_ids,
    "attention_mask": attention_mask
}

onnx_outputs = session.run(None, onnx_inputs)
print("🔎 ONNX Output Shape:", onnx_outputs[0].shape)


🔎 ONNX Output Shape: (1, 6, 768)


In [17]:
# ============================================================
# ✅ STEP 7: Compare PyTorch vs ONNX Outputs
# ============================================================


In [18]:
with torch.no_grad():
    torch_output = model(**inputs).last_hidden_state.cpu().numpy()

difference = ((onnx_outputs[0] - torch_output) ** 2).mean()
print(f"📊 Mean Squared Difference (PyTorch vs ONNX): {difference:.8f}")
print("🎯 Export completed and verified successfully!")


📊 Mean Squared Difference (PyTorch vs ONNX): 0.00000000
🎯 Export completed and verified successfully!
